# Tests for DataMux

In [1]:
import websockets
import logging
from serializers import Serializer

from typing import List

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
logging.basicConfig(format="%(asctime)-15s %(message)s", level=logging.INFO)
serializer = Serializer("json")
websocket = await websockets.connect('ws://127.0.0.1:3300/ws')

In [4]:
collection_arr: List[dict] = []
from pprint import pprint
async def list_collections():
    sendmsg = serializer.encode(b"list_collections", {})
    await websocket.send(sendmsg)
    recvmsg = await websocket.recv()
    topic, content = serializer.decode(recvmsg)
    print(topic)
    collections = content["collections"]
    print(*collections[:10], sep="\n")
    collection_arr.extend(collections)
await list_collections()

b'list_collections'
{'name': 'adhd_sin', 'description': 'Eye movements of participants during an audiovisual-SIN task', 'keywords': ['SIN', 'ADHD'], 'authors': [{'name': 'Anne M. P. Michalek', 'affiliation': 'Old Dominion University', 'email': 'aperrott@odu.edu'}], 'streams': {'gaze': {'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {}}, 'pupil': {'name': 'Pupil', 'description': 'pupil diameter', 'unit': 'mm', 'frequency': 60.0, 'fields': {'d': {'name': 'pupil d', 'description': 'Pupil Diameter', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp

In [5]:
collection = collection_arr[0]
streams_arr: List[dict] = []
async def list_collection_streams():
    sendmsg = serializer.encode(b"list_collection_streams", dict(collection_name=collection["name"]))
    await websocket.send(sendmsg)
    recvmsg = await websocket.recv()
    topic, content = serializer.decode(recvmsg)
    collection_name = content["collection_name"]
    print(topic, collection_name)
    streams = content["streams"]
    print(*streams[:10], sep="\n")
    streams_arr.extend(streams)
await list_collection_streams()

b'list_collection_streams' adhd_sin
{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'male', 'noise': '5', 'participant': '5', 'question': '18', 'collection': 'adhd_sin', 'id': 'gaze'}}
{'name': 'Pupil', 'description': 'pupil diameter', 'unit': 'mm', 'frequency': 60.0, 'fields': {'d': {'name': 'pupil d', 'description': 'Pupil Diameter', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', '

In [6]:
stream = streams_arr[0]
async def restream_streams():
    sendmsg = serializer.encode(
        b"restream_collection_stream",
        dict(
            collection_name=collection["name"],
            stream_name=stream["name"],
            attrs=stream["attrs"]
        )
    )
    await websocket.send(sendmsg)
    recvmsg = await websocket.recv()
    topic, content = serializer.decode(recvmsg)
    print(topic)
    print(content)
await restream_streams()

b'restream_collection_stream'
{'collection_name': 'adhd_sin', 'stream_name': 'Gaze', 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'male', 'noise': '5', 'participant': '5', 'question': '18', 'collection': 'adhd_sin', 'id': 'gaze'}, 'status': 1}


In [7]:
async def list_lsl_streams():
    sendmsg = serializer.encode(b"list_lsl_streams", {})
    await websocket.send(sendmsg)
    recvmsg = await websocket.recv()
    topic, content = serializer.decode(recvmsg)
    print(topic, content)
    streams = content["streams"]
    print(*streams, sep="\n")
    streams_arr.extend(streams)
await list_lsl_streams()

b'list_lsl_streams' {'streams': [{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'male', 'noise': '5', 'participant': '5', 'question': '18', 'collection': 'adhd_sin', 'id': 'gaze', 'dfds_mode': 'restream'}}]}
{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': '